In [1]:
#export
import k1lib, time, numpy as np, matplotlib.pyplot as plt

In [2]:
#export
class Callback:
    """Represents a callback. Define specific functions
    inside to intercept certain parts of the training
    loop. Can reference Learner's attrs directly
    using `self` like this:
    >>> self.opt # actually accessing Learner.opt, if `opt` is not defined
    
    You can also set Learner's attributes like this:
    >>> self.learner.xb = self.xb[None]
    This takes x batch of learner, unsqueeze it at
    the 0 position, then sets the x batch again.
    
    Normally, you will define a subclass of this and
    define specific intercept functions, but if you
    want to create a throwaway callback, then do this:
    >>> Callback().withCheckpoint("startRun", lambda: print("start running"))
    
    You can use `.cls` for a list of default Callback
    classes, for any particular needs"""
    def __init__(self, name=None):
        super(Callback, self).__init__()
        self.learner = None; self.cbs = None; self.suspended = False
        self.name = name or self.__class__.__name__
        self.order = 10 # can be modified by subclasses. A smaller order will be executed first
    def suspend(self): self.suspended = True
    def restore(self): self.suspended = False
    def __getattr__(self, attr):
        if attr == "learner": raise AttributeError(attr)
        if hasattr(self.learner, attr):
            return getattr(self.learner, attr)
        raise AttributeError(attr)
    def __getstate__(self):
        state = dict(self.__dict__)
        del state["learner"]; del state["cbs"]; return state
    def __setstate__(self, state): self.__dict__.update(state)
    def __repr__(self): return f"{self._reprHead}, can...\n{self._reprCan}"
    @property
    def _reprHead(self): return f"Callback `{self.name}`"
    @property
    def _reprCan(self): return """- cb.something: to get specific attribute "something" from learner if not available
- cb.withCheckpoint(checkpoint, f): to quickly insert an event handler
- cb.detach(): to remove itself from its parent Callbacks"""
    def withCheckpoint(self, checkpoint:str, f:callable): setattr(self, checkpoint, lambda: f(self)); return self
    def detach(self): self.cbs.remove(self.name); return self
Callback.cls = k1lib.Object()

In [3]:
#export
class Timings:
    @property
    def state(self):
        answer = dict(self.__dict__); answer.pop("getdoc", None); return answer
    @property
    def checkpoints(self):
        return [cp for cp in self.state if k1lib.isNumeric(self[cp])]
    def __getattr__(self, attr):
        if attr.startswith("_"): raise AttributeError()
        self.__dict__[attr] = 0; return 0
    def __getitem__(self, idx): return getattr(self, idx)
    def __setitem__(self, idx, value): setattr(self, idx, value)
    def plot(self):
        plt.figure(dpi=100); checkpoints = self.checkpoints
        timings = np.array([self[cp] for cp in checkpoints])
        maxTiming = timings.max()
        if maxTiming >= 1:
            plt.bar(checkpoints, timings); plt.ylabel("Time (s)")
        elif maxTiming >= 1e-3 and maxTiming < 1:
            plt.bar(checkpoints, timings*1e3); plt.ylabel("Time (ms)")
        elif maxTiming >= 1e-6 and maxTiming < 1e-3:
            plt.bar(checkpoints, timings*1e6); plt.ylabel("Time (us)")
        plt.xticks(rotation="vertical"); plt.show()
    def __repr__(self):
        cps = '\n'.join([f'- {cp}' for cp in self.checkpoints])
        return f"""Timings object. Checkpoints:\n{cps}\n
Can...
- t.startRun: to get specific checkpoint's execution time
- t.plot(): to plot all checkpoints"""

In [4]:
#export
class Callbacks:
    def __init__(self):
        self._learner = None; self.suspendStack = []; self.cbsDict = {}
        self.timings = Timings()
    @property
    def learner(self): return self._learner
    @learner.setter
    def learner(self, learner):
        self._learner = learner
        for cb in self.cbs: cb.learner = learner
    @property
    def cbs(self): return [*self.cbsDict.values()] # convenience method for looping over stuff
    def _sort(self):
        self.cbsDict = dict(sorted(self.cbsDict.items(), key=lambda x: x[1].order))
        return self
    def append(self, cb, name=None):
        """Adds a callback to the collection."""
        cb.learner = self.learner; cb.cbs = self
        name = name or cb.name
        if name in self.cbsDict:
            i = 0
            while f"{name}{i}" in self.cbsDict: i += 1
            name = f"{name}{i}"
        cb.name = name; self.cbsDict[name] = cb; self._sort(); return self
    def remove(self, name):
        """Removes a callback from the collection."""
        if name not in self.cbsDict: return print(f"Callback `{name}` not found")
        del self.cbsDict[name]; self._sort(); return self
    def removePrefix(self, prefix):
        """Removes any callback with a specific prefix"""
        for cb in self.cbs:
            if cb.name.startswith(prefix): self.remove(cb.name)
        return self
    def _handleCbCall(self, cb, checkpoint, *args, **kwargs):
        if hasattr(cb, checkpoint):
            return getattr(cb, checkpoint)(*args, **kwargs) != None
    def __call__(self, checkpoint, *args, **kwargs):
        beginTime = time.time()
        answer = any([self._handleCbCall(cb, checkpoint, *args, **kwargs) for cb in self.cbs])
        self.timings[checkpoint] += time.time() - beginTime
        return answer
    def __getattr__(self, attr):
        if attr == "cbsDict": raise AttributeError()
        if attr in self.cbsDict: return self.cbsDict[attr]
        else: raise AttributeError
    def __getstate__(self):
        state = dict(self.__dict__); del state["_learner"]; return state
    def __setstate__(self, state):
        self.__dict__.update(state)
        for cb in self.cbs: cb.cbs = self
    def __repr__(self):
        return "Callbacks:\n" + '\n'.join([f"- {cbName}" for cbName in self.cbsDict]) + """\n
Use...
- cbs.append(cb[, name]): to add a callback with a name
- cbs("startRun"): to trigger a specific event
- cbs.Loss: to get a specific callback
- cbs.timings: to get callback execution times
- cbs.suspend(["Loss", "Cuda"]): to temporarily prevent triggering events in
    specific callbacks. Can be used multiple times before restoring
- cbs.restore(): to restore latest temporary suspension
"""
    def withBasics(self):
        self.withProgressBar().withLoss().withDontTrainValid()
        return self.withCancelOnExplosion().withParamFinder()
    def withQOL(self):
        return self.withAutosave()
    def withAdvanced(self):
        return self.withLossLandscape().withHookModule().withHookParam()
    @staticmethod
    def standard(advanced=True):
        """Standard callbacks. Include advanced callbacks
        if `advanced` is True. They can impact performance,
        so it's possible to switch them off."""
        cbs = Callbacks().withProgressBar().withLoss().withLossLandscape()
        cbs.withDontTrainValid()
        if advanced: cbs.withHookModule().withHookParam()
        return cbs

In [5]:
#export
@k1lib.patch(Callbacks)
def suspend(self, cbsClasses:list):
    stackFrame = []
    for cb in self.cbs:
        for cbsClass in cbsClasses:
            if cb.__class__.__name__ == cbsClass:
                stackFrame.append(cb); break
    for cb in stackFrame: # do this cause if not, self.cbs will be modified while looping over it
        cb.suspend(); cb.detach()
    self.suspendStack.append(stackFrame)
@k1lib.patch(Callbacks)
def restore(self):
    stackFrame = self.suspendStack.pop()
    for cb in stackFrame:
        self.append(cb)
        cb.restore()

In [6]:
#export
from k1lib._callbacks import shorts as _shorts
from k1lib._callbacks import loss_progress as _loss_progress
from k1lib._callbacks import hookParam as _hookParam
from k1lib._callbacks import hookModule as _hookModule
from k1lib._callbacks import paramFinder as _paramFinder
from k1lib._callbacks import lossLandscape as _lossLandscape
from k1lib._callbacks import limits as _limits

In [7]:
!../export.py callbacks

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> bui